***Kaggle Titanic Competition***

Made by Rodolfo Amorim 

In [1]:
#Necessary Imports

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
from keras import *
import numpy as np 
import os as os
import keras

In [2]:
#Reading the csv input files

train_df = pd.read_csv(os.path.abspath('train.csv'))  
test_df = pd.read_csv(os.path.abspath('test.csv'))  

In [13]:
#Cleaning of the input data

train_df2= train_df.drop(['PassengerId','Name','Ticket','Cabin','Embarked'], axis=1)
values={'Age':np.mean(train_df2.iloc[:,3])}
train_df2 = train_df2.fillna(value=values) #Filling in the NaN entries by the mean of the ages of the titanic passangers

#Number of NaN's to check if the previous step was correct
#nan_count = train_df2.isna().sum()
#print(nan_count)
#print('\n')

y_train = train_df2.iloc[:,0] #True outcomes
x_train = train_df2.drop(['Survived'],axis=1) #Features

x_train = pd.get_dummies(x_train, columns=['Sex'], drop_first=True,dtype=float) #One-hot encode the 'Sex' column 
x_train.iloc[:,0] = x_train.iloc[:,0].astype(float)

#Rescaling the columns of the dataframe
x_train.iloc[:,1] = x_train.iloc[:,1]/np.max(x_train.iloc[:,1])
x_train.iloc[:,0] = x_train.iloc[:,0]/3 
x_train.iloc[:,4] = x_train.iloc[:,4]/np.max(x_train.iloc[:,4]) 

print('Number of Samples in x_train: ', len(x_train))
print('Number of features in x_train: ', len(x_train.columns))

Number of Samples in x_train:  891
Number of features in x_train:  6


In [15]:
#Cleaning of the x_test data

teste_df2= test_df.drop(['PassengerId','Name','Ticket','Cabin','Embarked'], axis=1)
values={'Age':np.mean(teste_df2.iloc[:,2]),'Fare':np.mean(teste_df2.iloc[:,5]) }
teste_df2 = teste_df2.fillna(value=values) #Filling in the NaN entries by the mean of the ages and fares of the titanic passangers
passengerID = test_df.iloc[:,0]

x_test = teste_df2  #Features
x_test = pd.get_dummies(x_test, columns=['Sex'], drop_first=True,dtype=float) #One-hot encode the 'Sex' column
x_test.iloc[:,0] = x_test.iloc[:,0].astype(float)

#Rescaling the columns of the dataframe
x_test.iloc[:,1] = x_test.iloc[:,1]/np.max(x_test.iloc[:,1])
x_test.iloc[:,0] = x_test.iloc[:,0]/3 
x_test.iloc[:,4] = x_test.iloc[:,4]/np.max(x_test.iloc[:,4])

print('Number of Samples in x_test: ', len(x_test))

Number of Samples in x_test:  418


**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier


param_grid = { 
    'n_estimators': [100,200],
    'max_features': ['auto', 'sqrt','log2'],
    'max_depth' : [8,9,10],
    'max_leaf_nodes': [3, 6],
    'criterion' :['gini','entropy','log_loss']
}

#Performing gridsearch over all the possible Random Forest hyperparameters
CVr = GridSearchCV(estimator= RandomForestClassifier(random_state=1), param_grid = param_grid, cv = 5)
CVr.fit(x_train, y_train)

In [32]:
#Obtaining the output and saving it to a submission csv

model = RandomForestClassifier(random_state=1, n_estimators=CVr.best_params_['n_estimators'],max_features=CVr.best_params_['max_features'],max_depth=CVr.best_params_['max_depth'],criterion=CVr.best_params_['criterion'], max_leaf_nodes=CVr.best_params_['max_leaf_nodes'])
model.fit(x_train, y_train)
predictions = model.predict(x_test) #Survival predictions


pred_df = pd.DataFrame(predictions, columns=['Survived'])
prediction_df = pred_df.set_index(passengerID)
prediction_df.to_csv('submission.csv',header=['Survived']) #Making the csv file